<a href="https://colab.research.google.com/github/Devisrinerusu/Certificates/blob/main/Movie_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.listdir('/content/')


['.config', 'sample_data']

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Load dataset
file_path = " //content/movies_data.csv"  # Update if needed
df = pd.read_csv(f"/content/movies_data.csv")

# Drop missing values in required columns
df.dropna(subset=["Name", "Genre", "Year", "Rating", "Votes"], inplace=True)

# Normalize movie names
df["Name"] = df["Name"].str.strip().str.lower()

# Convert Year, Rating, and Votes to numeric
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
df["Votes"] = pd.to_numeric(df["Votes"], errors="coerce")

# Drop invalid rows
df.dropna(inplace=True)

# Convert Year to integer
df["Year"] = df["Year"].astype(int)

# Encode genres
label_encoder = LabelEncoder()
df["Genre_Label"] = label_encoder.fit_transform(df["Genre"])

# Naïve Bayes Model for Genre Classification
nb_pipeline = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("nb", MultinomialNB())
])
nb_pipeline.fit(df["Genre"], df["Genre_Label"])

#  Recommendation Function Based on Genre, Rating, and Votes
def recommend_movies_naive_bayes():
    movie_name = input("Enter a movie name: ").strip().lower()  # Take user input

    if movie_name not in df["Name"].values:
        return "Movie not found in dataset."

    # Get movie details
    movie_row = df[df["Name"] == movie_name].iloc[0]
    movie_genre = movie_row["Genre"]
    movie_year = movie_row["Year"]

    # Predict genre using Naïve Bayes
    predicted_genre_label = nb_pipeline.predict([movie_genre])[0]
    predicted_genre = label_encoder.inverse_transform([predicted_genre_label])[0]

    # Filter movies of the same genre
    genre_movies = df[df["Genre"] == predicted_genre].copy()

    # Sort by Rating (highest first) and then by Votes (highest first)
    genre_movies = genre_movies.sort_values(by=["Rating", "Votes"], ascending=[False, False])

    # Select top 5 recommendations
    similar_movies = genre_movies.head(5)[["Name", "Genre", "Year", "Rating", "Votes"]]

    print("\nRecommended Movies:")
    print(similar_movies.to_string(index=False))

# Call the function to get user input and show recommendations
recommend_movies_naive_bayes()


Enter a movie name: 100% LOVE

Recommended Movies:
                  Name                  Genre  Year  Rating  Votes
          love sorries Comedy, Drama, Romance  2021     9.3     79
         chupke chupke Comedy, Drama, Romance  1975     8.3  11923
jo jeeta wohi sikandar Comedy, Drama, Romance  1992     8.2  22147
                barfi! Comedy, Drama, Romance  2012     8.1  77377
        dil chahta hai Comedy, Drama, Romance  2001     8.1  68138
